<a href="https://colab.research.google.com/github/PERSONAL-PROJECTS-2000/SMART-STUDY-ASSISTANT/blob/main/SMART_STUDY_ASSISTANT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from google.colab import userdata
from google import genai
GEMINI_API_KEY=userdata.get('GOOGLE_API_KEY')
client=genai.Client(api_key=GEMINI_API_KEY)

%pip install -q -U google-generativeai gradio python-dotenv pydantic docx2txt pandas openpyxl pillow
%pip install -q unstructured
%pip install pdfplumber

import os
import time
import pandas as pd
import gradio as gr
from PIL import Image
from datetime import datetime
from io import BytesIO
from typing import List, Dict, Optional
import tempfile
import pdfplumber
import docx2txt
import json
from google import genai
from google.genai import types

##########
import os
import time
import pandas as pd
import gradio as gr
from PIL import Image
from datetime import datetime,timedelta
from typing import List,Dict,Optional,Union
from google import genai
from google.genai import types

try:
    if 'google.colab' in str(get_ipython()):
        from google.colab import userdata
        API_KEY=userdata.get('GOOGLE_API_KEY')
    else:
        API_KEY=os.getenv('GEMINI_API_KEY')
    if not API_KEY:
        raise ValueError("API Key not found. Please set 'GOOGLE_API_KEY' in Colab Secrets or environment.")
    client=genai.Client(api_key=API_KEY)
except Exception as e:
    print(f"Initialization Error: {e}")
    client=None

MODEL_NAME_FAST = "gemini-2.5-flash"
MODEL_NAME_PRO = "gemini-2.5-pro"
SYSTEM_INSTRUCTION_TUTOR = "You are a clear, patient, and highly helpful AI-Powered Study Buddy and Tutor. Your goal is to simplify complex concepts, summarize notes, and create helpful study materials. Respond conversationally and encourage the student."
SYSTEM_INSTRUCTION_PLANNER = "You are a professional study planner. Generate a suitable timetable in a clean, Markdown-formatted table. The output MUST be a Markdown table with three columns: Time Slot, Task Name, and Duration."
SUPPORTED_FILE_TYPES = [
    ".txt", ".pdf", ".doc", ".docx", ".pptx", ".xlsx",
    ".jpg", ".jpeg", ".png"
]

##########
MODEL_NAME_FAST="gemini-2.5-flash"
SYSTEM_INSTRUCTION="You are an AI-Powered Study Buddy. You provide clear, concise, and structured responses. When processing documents, focus solely on the content and task provided. For explanations, maintain an educational and encouraging tone."
SUPPORTED_FILE_TYPES = [".txt", ".pdf", ".doc", ".docx", ".pptx", ".xlsx",".jpg", ".jpeg", ".png"]

def upload_file_to_gemini(file_path):
    """Uploads a local file to the Gemini API's File Service."""
    if not file_path:
        return None
    try:
        print(f"Uploading file: {file_path}")
        uploaded_file = client.files.upload(file=file_path)
        return uploaded_file
    except Exception as e:
        print(f"File upload error: {e}")
        return None

##########
def upload_file_to_gemini(file_path):
    if not file_path:
        return None
    print(f"Uploading file: {file_path}")
    uploaded_file=client.files.upload(file=file_path)
    return uploaded_file

def generate_content_response(contents, system_prompt=SYSTEM_INSTRUCTION_TUTOR, model=MODEL_NAME_FAST, tool_config=None):
    """Central function to call the Gemini API."""
    if not client:
        return "AI Client not initialized. Please check your API Key configuration."

    config = types.GenerateContentConfig(
        system_instruction=system_prompt,
        tools=tool_config if tool_config else None
    )

    try:
        response = client.models.generate_content(
            model=model,
            contents=contents,
            config=config
        )
        # Check for tool usage to return full response (for search features)
        if response.candidates and response.candidates[0].tools:
             return response.text
        return response.text
    except Exception as e:
        print(f"API Call Error: {e}")
        return f"Sorry, an error occurred during the AI task: {e}"

##########
def generate_content_response(contents,system_prompt=SYSTEM_INSTRUCTION, tool_config=None):
    if not client:
        return "AI Client not initialized. Please check your API Key configuration."
    config=types.GenerateContentConfig(system_instruction=system_prompt,tools=tool_config if tool_config else None)
    response=client.models.generate_content(model=MODEL_NAME_FAST,contents=contents,config=config)
    return response.text

def get_file_preview(file_obj):
    if file_obj and file_obj.name:
        path = file_obj.name
        if path.lower().endswith(('.png', '.jpg', '.jpeg')):
            return path
        elif path.lower().endswith('.pdf'):
            try:
                # Extract first page as image
                with pdfplumber.open(path) as pdf:
                    first_page = pdf.pages[0]
                    im = first_page.to_image(resolution=72)

                    # Save to a temporary file for Gradio to display
                    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp_file:
                        im.save(tmp_file.name)
                        return tmp_file.name
            except Exception as e:
                print(f"PDF Preview Error: {e}")
                return None
        # For other types, return None or a placeholder
        return None
    return None

def handle_chat_prompt(message: str, history: List):
    """Handles the simple text input chat prompt using the ChatService."""
    if not client:
        return "AI Client not initialized."

    # We will use the chat service for better memory management
    chat = client.chats.create(
        model=MODEL_NAME_FAST,
        history=history,
        system_instruction=SYSTEM_INSTRUCTION_TUTOR
    )

    # Send the new message
    response = chat.send_message(message)

    # Return the response text
    return response.text

##########
def handle_chat_prompt(text_prompt):
    if not text_prompt:
        return "Please enter a question or command."
    return generate_content_response([text_prompt])

def document_task_handler(file_obj, topic_name, task_prompt_template, skill_level=None):
    """Generic handler for all document-based tasks."""
    if not file_obj:
        return "Please upload a document or image first."

    # 1. Upload file
    uploaded_file = upload_file_to_gemini(file_obj.name)
    if not uploaded_file:
        return "Error uploading file to Gemini service."

    # 2. Construct the prompt
    topic_clause = f" on the specific topic: '{topic_name}'" if topic_name else ""
    level_clause = f" for a {skill_level} student" if skill_level else ""

    prompt = task_prompt_template.format(topic=topic_clause, level=level_clause)

    # 3. Call the API
    contents = [
        uploaded_file,
        prompt
    ]

    try:
        response_text = generate_content_response(contents)
        return response_text
    finally:
        # Clean up the file from the service (recommended)
        client.files.delete(name=uploaded_file.name)

##########
def document_task(file_obj,topic_name,task_prompt_template,skill_level=None):
    if not file_obj:
        return "Please upload a document first."
    uploaded_file=upload_file_to_gemini(file_obj.name)
    if not uploaded_file:
        return "Error uploading file."
    topic_clause=f" on the topic: '{topic_name}'" if topic_name else ""
    level_clause=f" for a {skill_level} student" if skill_level else ""
    prompt=task_prompt_template.format(topic=topic_clause, level=level_clause)
    contents=[uploaded_file,prompt]
    try:
        response_text=generate_content_response(contents,system_prompt=SYSTEM_INSTRUCTION)
        return response_text
    finally:
        client.files.delete(name=uploaded_file.name)

Analyze the document/image, read the content (including handwriting/OCR if present) and identify the main topic of the uploaded document/image, considering the optional topic name '{topic_name if topic_name else 'none'}. Then

def summarize_doc(file_obj, topic_name):
    prompt_template = f"Analyze the document/image, read the content (including handwriting/OCR if present) and identify the main topic of the uploaded document/image, considering the optional topic name '{topic_name if topic_name else 'none'}. Then generate a comprehensive summary of the uploaded document {topic_name}. Use bullet points and clear language."
    return document_task_handler(file_obj, topic_name, prompt_template)

##########
def summarize_doc(file_obj,topic_name):
    prompt_template="Generate a comprehensive summary of the uploaded document{topic}. Use bullet points for clarity."
    return document_task(file_obj,topic_name,prompt_template)

def explain_doc(file_obj, topic_name, skill_level):
    prompt_template = f"Analyze the document/image, read the content (including handwriting/OCR if present) and identify the main topic of the uploaded document/image, considering the optional topic name '{topic_name if topic_name else 'none'}. Then provide a detailed explanation of the uploaded document {topic_name} and content {skill_level}. Use analogies and clear examples suitable for that level."
    return document_task_handler(file_obj, topic_name, prompt_template, skill_level=skill_level)

##########
def explain_doc(file_obj,topic_name,skill_level):
    prompt_template="Provide a detailed explanation of the uploaded document{topic} and content{level}. Use analogies and clear examples."
    return document_task(file_obj,topic_name,prompt_template,skill_level=skill_level)

def find_keywords(file_obj, topic_name):
    prompt_template = f"Analyze the document/image, read the content (including handwriting/OCR if present) and identify the main topic of the uploaded document/image, considering the optional topic name '{topic_name if topic_name else 'none'}. Then carefully select 5-20 key words from the uploaded document {topic_name}. For each keyword, provide a simple definition and a brief, real-world example."
    return document_task_handler(file_obj, topic_name, prompt_template)

##########
def find_keywords(file_obj,topic_name):
    prompt_template="Carefully select 5-7 key words from the uploaded document{topic}. For each keyword, provide a simple definition and a brief, real-world example."
    return document_task(file_obj,topic_name,prompt_template)

def generate_answers(file_obj, topic_name):
    prompt_template = f"The uploaded document contains a list of questions {topic_name}. Analyze the document, read the questions and their numbering (including handwriting/OCR if present), identify the main topic of the uploaded document/image, considering the optional topic name '{topic_name if topic_name else 'none'}', and formulate a search query for each question to find a complete and accurate answer on the web. Provide a complete and accurate answer for each question presented in the document along with the respective corresponding numbering. Format the response clearly with numbered questions and answers."
    return document_task_handler(file_obj, topic_name, prompt_template)

##########
def generate_answers(file_obj,topic_name):
    prompt_template="The uploaded document contains a list of questions{topic}. Provide a complete and accurate answer for each question presented in the document."
    return document_task(file_obj,topic_name,prompt_template)

def find_similar_content_or_image(file_obj, topic_name):
    """Finds similar text/document or image using Google Search tool."""
    if not file_obj:
        return "Please upload a document or image first."

    uploaded_file = upload_file_to_gemini(file_obj.name)
    if not uploaded_file:
        return "Error uploading file to Gemini service."

    mime_type = uploaded_file.mime_type

    if 'image' in mime_type:
        prompt = f"Analyze the document, read the content (including handwriting/OCR if present) and formulate a search query to find a similar/closely-related image as this uploaded image (on topic: {topic_name if topic_name else 'general'}). Provide the direct image URL and a brief description. Do not output anything else."
    else:
        prompt = f"Analyze the document, read the content (including handwriting/OCR if present) and formulate a search query to find a document or article with similar/closely-related text and information as this uploaded document (on topic: {topic_name if topic_name else 'general'}). Provide a direct link (URL) and a 1-sentence description of the link."

    contents = [uploaded_file, prompt]
    tool_config = [types.Tool(google_search=types.GoogleSearch(type="google_search"))]

    try:
        response_text = generate_content_response(contents, tool_config=tool_config)
        return response_text
    finally:
        client.files.delete(name=uploaded_file.name)

##########
def find_similar_content(file_obj,topic_name):
    if not file_obj:
        return "Please upload a document or image first."
    uploaded_file=upload_file_to_gemini(file_obj.name)
    if not uploaded_file:
        return "Error uploading file."
    mime_type=uploaded_file.mime_type
    if 'image' in mime_type:
        prompt="Search the web for a similar-looking image and provide the image link (URL) only."
    else:
        prompt=f"Search the web for a document or article with similar/closely-related text and information as this uploaded document (on topic: {topic_name if topic_name else 'general'}). Provide a direct link (URL) and a 1-sentence description of the link."
    contents=[uploaded_file, prompt]
    tool_config=[types.Tool(google_search=types.GoogleSearch(type="google_search"))]
    try:
        response_text=generate_content_response(contents,tool_config=tool_config)
        return response_text
    finally:
        client.files.delete(name=uploaded_file.name)

def find_topic_links(file_obj, topic_name):
    """Searches the web for documents/sources related to the topic/content."""
    if not file_obj:
        return "Please upload a document first."

    uploaded_file = upload_file_to_gemini(file_obj.name)
    if not uploaded_file:
        return "Error uploading file to Gemini service."

    prompt = f"Identify the main topic of the uploaded document/image, considering the optional topic name '{topic_name if topic_name else 'none'}'. Analyze the document, read the content (including handwriting/OCR if present) and formulate a search query to find at least 5 reliable, relevant links to documentations or information sources on that specific topic. List the links and their titles clearly. Output should be ONLY the links and descriptions."
    contents = [uploaded_file, prompt]

    tool_config = [types.Tool(google_search=types.GoogleSearch(type="google_search"))]

    try:
        response_text = generate_content_response(contents, tool_config=tool_config)
        return response_text
    finally:
        client.files.delete(name=uploaded_file.name)

##########
def find_topic_links(file_obj,topic_name):
    if not file_obj:
        return "Please upload a document first."
    uploaded_file=upload_file_to_gemini(file_obj.name)
    if not uploaded_file:
        return "Error uploading file."
    prompt=f"Identify the main topic of the uploaded document, considering the optional topic name '{topic_name if topic_name else 'none'}'. Then, use the search tool to find at least 5 reliable, relevant links to documentations or information sources on that specific topic. List the links and their titles clearly."
    contents=[uploaded_file, prompt]
    tool_config=[types.Tool(google_search=types.GoogleSearch(type="google_search"))]
    try:
        response_text=generate_content_response(contents, tool_config=tool_config)
        return response_text
    finally:
        client.files.delete(name=uploaded_file.name)

def find_image_by_content(file_obj_path, topic_name):
    """
    Analyzes the uploaded file (document/image, including handwriting)
    and searches the web for a closely related image.
    """
    if not file_obj_path:
        return "Please upload a document or image first in the 'Upload Document/Image' field."

    uploaded_file = upload_file_to_gemini(file_obj_path)
    if not uploaded_file:
        return "Error uploading file to Gemini service."

    topic_clause = f" on the specific topic: '{topic_name}'" if topic_name else ""

    prompt = f"""
    Analyze the uploaded document or image{topic_clause}.
    If it is a document, read the content (including handwriting/OCR if present) and formulate a search query to find a high-quality, closely related image on the web.
    If it is an image, find a similar or highly relevant image on the web.
    Use the Google Search tool to find the image.
    Provide the **direct image URL** and a brief, 1-sentence description of the image found.
    Output ONLY the URL and description.
    """

    contents = [uploaded_file, prompt]
    tool_config = [types.Tool(google_search=types.GoogleSearch(type="google_search"))]

    try:
        response_text = generate_content_response(contents, tool_config=tool_config)
        return response_text
    finally:
        client.files.delete(name=uploaded_file.name)

##########
def find_image(text_prompt):
    if not text_prompt:
        return "Please enter a description for the image you are looking for."
    prompt=f"Find a high-quality image that fits this description: '{text_prompt}'. Provide the direct image URL and a brief description of the image."
    tool_config=[types.Tool(google_search=types.GoogleSearch(type="google_search"))]
    response=generate_content_response([prompt], tool_config=tool_config)
    if "http" in response:
        import re
        url_match=re.search(r'(https?://[^\s)"]+)',response)
        image_url=url_match.group(0) if url_match else None
        if image_url:
            return gr.Image(value=image_url,label="Found Image",height=300)
    return response

def time_to_seconds(value, unit):
    if unit == "seconds":
        return value
    elif unit == "minutes":
        return value * 60
    elif unit == "hours":
        return value * 3600
    return 0

def generate_timetable(tasks_df, estimated_total_time, total_time_unit):
    """Generates a timetable based on user-defined tasks and time constraints."""

    # Convert DataFrame to list of dictionaries
    tasks_data = tasks_df.to_dict('records')

    if not tasks_data or not estimated_total_time:
        return "Please enter at least one task and the Estimated Total Time."

    tasks = []
    for row in tasks_data:
        task_name = row.get('Task Name', '').strip()
        from_time = row.get('From (DateTime)', '')
        to_time = row.get('To (DateTime)', '')
        estimated_time = row.get('Estimated Duration (Text)', '')

        if task_name:
            tasks.append({
                'name': task_name,
                'from': from_time,
                'to': to_time,
                'estimate': estimated_time
            })

    if not tasks:
        return "Please ensure you have entered a 'Task Name' for at least one row."

    # Format the data for the LLM
    task_list_str = "\n".join([
        f"- Task: {t['name']}, Start: {t['from'] or 'N/A'}, End: {t['to'] or 'N/A'}, Estimate: {t['estimate'] or 'N/A'}"
        for t in tasks
    ])

    prompt = f"""
    Generate a suitable study timetable based on the following information.
    The total time available for the schedule is **{estimated_total_time} {total_time_unit}**.

    **Tasks to Schedule:**
    {task_list_str}

    **Instructions:**
    1. The output MUST be a clean, Markdown-formatted table with three columns: **Time Slot**, **Task Name**, and **Duration**.
    2. Fill in the 'Time Slot' based on a logical flow starting now, or use the explicit 'From'/'To' times provided.
    3. Ensure the schedule fits reasonably within the Estimated Total Time.
    """

    # Use the more capable PRO model for complex planning/reasoning
    response_text = generate_content_response(
        [prompt],
        system_prompt=SYSTEM_INSTRUCTION_PLANNER,
        model=MODEL_NAME_PRO
    )

    return response_text

##########
def generate_timetable(tasks_data,estimated_total_time,total_time_unit):
    if not tasks_data or not estimated_total_time:
        return "Please enter at least one task and the Estimated Total Time."
    tasks=[]
    for row in tasks_data:
        task_name=row.get('Task name','').strip()
        from_time=row.get('From (date and time input)','')
        to_time=row.get('To (date and time input)','')
        estimated_time=row.get('Estimated time to complete the task','')
        if task_name:
            tasks.append({
                'name': task_name,
                'from': from_time,
                'to': to_time,
                'estimate': estimated_time
            })
    if not tasks:
        return "Please ensure you have entered a 'Task name' for at least one row."
    task_list_str="\n".join([
        f"- Task: {t['name']}, Start: {t['from'] or 'N/A'}, End: {t['to'] or 'N/A'}, Estimate: {t['estimate'] or 'N/A'}"
        for t in tasks
    ])
    prompt=f"""
    You are a professional study planner. Generate a suitable timetable in a clean, Markdown-formatted table.
    The total time available for the schedule is **{estimated_total_time} {total_time_unit}**.
    **Tasks to Schedule:**
    {task_list_str}
    **Instructions:**
    1. Study the tasks, existing timings, and estimated times.
    2. Create a schedule that fits within the Estimated Total Time.
    3. The output MUST be a Markdown table with three columns: **Time Slot**, **Task Name**, and **Duration**.
    4. Fill in the 'Time Slot' based on a logical flow starting from the current time, or use the 'From'/'To' if provided.
    5. If 'From' and 'To' times are provided, prioritize fitting those tasks first.
    """
    response_text=generate_content_response([prompt],system_prompt="You are a professional study planner and your output must be a Markdown table.")
    return response_text

def clear_file_and_state():
    """Clears the file input and the stored file state."""
    return None, None # Clears file_input and file_state_path

# The ChatService needs to be initialized outside the function for persistent history
def initialize_chat():
    return client.chats.create(
        model=MODEL_NAME_FAST,
        history=[],
        system_instruction=SYSTEM_INSTRUCTION_TUTOR
    )

with gr.Blocks(theme=gr.themes.Soft(), title="SMART STUDY ASSISTANT", mode="dark") as demo:

    # --- State Variables ---
    # Used to pass data between components reliably, especially file path
    file_state_path = gr.State(None)
    chat_history = gr.State([]) # To store and maintain chat history

    # --- Header and Night Mode Toggle ---
    with gr.Row(variant="panel"):
        gr.Markdown("# 💡 SMART STUDY ASSISTANT")

        # Checkbox for Night Mode (placed top right)
        night_mode_toggle = gr.Checkbox(
            label="Night",
            value=False,
            interactive=True,
            scale=1
        )

    # JS to handle the dark mode toggle
    demo.load(js="""
        const toggle = document.getElementById('dark-mode-toggle');
        if (toggle) {
            toggle.onchange = function() {
                if (this.checked) {
                    document.body.classList.add('dark');
                } else {
                    document.body.classList.remove('dark');
                }
            };
        }
    """)

    # --- Main Tabs ---
    with gr.Tabs() as main_tabs:

        # 1. Quick Tutor (Chat)
        with gr.TabItem("Quick Tutor"):

            chat_interface = gr.ChatInterface(
                fn=handle_chat_prompt,
                chatbot=gr.Chatbot(
                    height=450,
                    #show_copy_button=True,
                    label="AI Study Buddy Chat"
                ),
                textbox=gr.Textbox(
                    placeholder="Ask a question, define a term, or request a brief explanation.",
                    container=False,
                    label="Your Prompt Input"
                ),
                submit_btn="Send"
                # retry_btn="Try Again", # Removed as not a valid parameter
                # undo_btn="Undo Last", # Removed as not a valid parameter
                # clear_btn="Clear Chat" # Removed as not a valid parameter
            )


        # 2. Document Analyzer
        with gr.TabItem("Document Analyzer"):

            doc_output = gr.Markdown(
                "Upload a document or image and select a task tab below.",
                label="Analysis Result"
            )

            with gr.Row(variant="panel"):
                # File Upload Input (Displays first page/image)
                file_input = gr.File(
                    label="Upload Document/Image (.txt, .pdf, .docx, .pptx, .jpg, etc.)",
                    file_count="single",
                    file_types=SUPPORTED_FILE_TYPES,
                    height=100
                )

                # Update file_state_path and preview on upload
                file_input.upload(
                    fn=lambda f: (f.name, get_file_preview(f)),
                    inputs=[file_input],
                    outputs=[file_state_path, file_input]
                )

                with gr.Column(scale=1):
                    # Topic Name Input
                    topic_input = gr.Textbox(
                        placeholder="Enter the topic name (optional)",
                        label="Topic Name (Optional)"
                    )

                    # Clear Button (Custom logic to clear file_input and state)
                    gr.Button("Clear Upload").click(
                        fn=clear_file_and_state,
                        inputs=[],
                        outputs=[file_input, file_state_path]
                    )

            # Sub-tabs for the 7 features
            with gr.Tabs() as doc_tabs:

                # Tab 1: Summarize
                with gr.TabItem("Summarize"):
                    gr.Button("Summarize", variant="primary").click(
                        fn=summarize_doc,
                        inputs=[file_state_path, topic_input],
                        outputs=doc_output
                    )

                # Tab 2: Explain by Level
                with gr.TabItem("Explain"):
                    gr.Markdown("Select the required explanation level:")
                    with gr.Row() as explanation_level_buttons:
                        gr.Button("for beginner level").click(
                            fn=lambda f, t: explain_doc(f, t, "beginner"),
                            inputs=[file_state_path, topic_input],
                            outputs=doc_output
                        )
                        gr.Button("for intermediate level").click(
                            fn=lambda f, t: explain_doc(f, t, "intermediate"),
                            inputs=[file_state_path, topic_input],
                            outputs=doc_output
                        )
                        gr.Button("for advanced level").click(
                            fn=lambda f, t: explain_doc(f, t, "advanced"),
                            inputs=[file_state_path, topic_input],
                            outputs=doc_output
                        )

                # Tab 3: Key Words
                with gr.TabItem("Key words"):
                    gr.Button("Find and explain key words", variant="primary").click(
                        fn=find_keywords,
                        inputs=[file_state_path, topic_input],
                        outputs=doc_output
                    )

                # Tab 4: Similar Document/Image
                with gr.TabItem("Similar document/image"):
                    gr.Button("Find similar document/image", variant="primary").click(
                        fn=find_similar_content_or_image,
                        inputs=[file_state_path, topic_input],
                        outputs=doc_output
                    )

                # Tab 5: Related Documentation on the Web
                with gr.TabItem("Related documentation on the web"):
                    gr.Button("Find links", variant="primary").click(
                        fn=find_topic_links,
                        inputs=[file_state_path, topic_input],
                        outputs=doc_output
                    )

                # Tab 6: Generate Answers
                with gr.TabItem("Generate answers"):
                    gr.Button("Generate answers", variant="primary").click(
                        fn=generate_answers,
                        inputs=[file_state_path, topic_input],
                        outputs=doc_output
                    )

                # Tab 7: Find Image
                with gr.TabItem("Find image"):
                    # Use a separate text box for the image search prompt
                    image_prompt_input = gr.Textbox(
                        placeholder="Enter the description of the image you need to search for.",
                        label="Image Description Input"
                    )
                    gr.Button("Find image", variant="primary").click(
                        fn=find_image_by_content,
                        inputs=[image_prompt_input], # This was file_state_path but the function expects file_obj_path
                        outputs=doc_output
                    )

        # 3. Timetable Generator
        with gr.TabItem("Timetable Generator"):

            with gr.Row():
                # Task Input Table (Changed to column-based structure as requested)
                gr.Markdown("### Task List")

                # Define columns structure for the Dataframe
                task_df_headers = [
                    "Task Name",
                    "From (DateTime)",
                    "To (DateTime)",
                    "Estimated Duration (Text)"
                ]

                # Use Textbox for all date/time fields in the DataFrame for flexibility with 'optional'
                # And use a custom component for the final display.
                task_df = gr.Dataframe(
                    headers=task_df_headers,
                    datatype=["str", "str", "str", "str"],
                    col_count=4,
                    row_count=1,
                    label="Enter Task Details Below (Click cells to edit)",
                    interactive=True
                )

            with gr.Row():
                # Button to add rows (Easier and more common than 'Add Column' in Dataframes)
                gr.Button("Add Row", variant="secondary").click(
                    fn=lambda df: pd.concat([df, pd.DataFrame([["", "", "", ""]], columns=df.columns)]),
                    inputs=[task_df],
                    outputs=[task_df]
                )

                gr.Button("Clear Tasks", variant="secondary").click(
                    fn=lambda: pd.DataFrame(columns=task_df_headers, data=[["", "", "", ""]]),
                    inputs=[],
                    outputs=[task_df]
                )

            with gr.Row():
                estimated_total_time = gr.Number(label="Estimated Total Time (e.g., 4)", value=4)
                total_time_unit = gr.Radio(["seconds", "minutes", "hours"], label="Unit", value="hours")

            timetable_output = gr.Markdown("Click 'Generate timetable' to see your schedule.", label="Generated Timetable")

            # Generate and Download Button Row
            with gr.Row():
                generate_btn = gr.Button("Generate timetable", variant="primary")
                download_btn = gr.Button("Download Timetable (.xlsx)", variant="secondary", interactive=False)

            # Action for Generate Button
            generate_btn.click(
                fn=generate_timetable,
                inputs=[task_df, estimated_total_time, total_time_unit],
                outputs=[timetable_output]
            )

# --- LAUNCH THE APP ---
if __name__ == "__main__":
    demo.launch(share=True)

##########
with gr.Blocks(theme=gr.themes.Soft(), title="AI-Powered Study Buddy") as demo:
    gr.HTML("""
        <div style="display: flex; justify-content: space-between; align-items: center;">
            <h1 style="margin: 0; padding: 10px 0;">📚 AI-Powered Study Buddy (Gemini)</h1>
            <label class="gradio-checkbox-label" style="font-size: 1.1em; cursor: pointer;">
                <input type="checkbox" id="dark-mode-toggle" onchange="document.body.classList.toggle('dark');" />
                <span class="gradio-checkbox-mark"></span>
                Day/Night Mode
            </label>
        </div>
        <script>
            if (window.matchMedia && window.matchMedia('(prefers-color-scheme: dark)').matches) {
                document.getElementById('dark-mode-toggle').checked = true;
                document.body.classList.add('dark');
            }
        </script>
    """)
    with gr.Tabs():
        with gr.TabItem("💬 Quick Tutor"):
            chat_interface=gr.ChatInterface(
                fn=handle_chat_prompt,
                chatbot=gr.Chatbot(height=450),
                textbox=gr.Textbox(placeholder="Ask a question, define a term, or request a brief explanation.", container=False),
                title="Ask the AI Tutor Anything",
                submit_btn="Send"
            )
        with gr.TabItem("📄 Document Analyzer"):
            doc_output=gr.Markdown(
                "Upload a document and select an analysis task.",
                label="Analysis Result"
            )
            with gr.Row():
                file_input = gr.File(
                    label="Upload Document/Image",
                    file_count="single",
                    file_types=SUPPORTED_FILE_TYPES,
                    height=100
                )
                topic_input=gr.Textbox(
                    placeholder="Enter the topic name (optional, helps focus the AI)",
                    label="Topic Name (Optional)"
                )
                gr.Markdown("### 1. Document Content Analysis")
                with gr.Row():
                    gr.Button("Summarize").click(
                        summarize_doc,
                        inputs=[file_input, topic_input],
                        outputs=doc_output
                    )
                    gr.Button("Find and explain key words").click(
                        find_keywords,
                        inputs=[file_input, topic_input],
                        outputs=doc_output
                    )
                    gr.Button("Generate answers (if document has questions)").click(
                        generate_answers,
                        inputs=[file_input, topic_input],
                        outputs=doc_output
                    )
                gr.Markdown("### 2. Explain Content by Level")
                with gr.Row():
                    gr.Button("Explain for beginner level").click(
                        lambda f, t: explain_doc(f, t, "beginner"),
                        inputs=[file_input, topic_input],
                        outputs=doc_output
                    )
                    gr.Button("Explain for intermediate level").click(
                        lambda f, t: explain_doc(f, t, "intermediate"),
                        inputs=[file_input, topic_input],
                        outputs=doc_output
                    )
                    gr.Button("Explain for advanced level").click(
                        lambda f, t: explain_doc(f, t, "advanced"),
                        inputs=[file_input, topic_input],
                        outputs=doc_output
                    )
                gr.Markdown("### 3. Web Search & Image Tools")
                gr.Textbox(
                    placeholder="Enter image description here (e.g., 'diagram of a solar flare')",
                    label="Image Search Prompt",
                    lines=1
                ).submit(
                    find_image,
                    inputs=gr.Textbox(type='text'),
                    outputs=doc_output
                ).then(
                    None, None, None
                )
                with gr.Row():
                    gr.Button("Find similar document/image").click(
                        find_similar_content,
                        inputs=[file_input, topic_input],
                        outputs=doc_output
                    )
                    gr.Button("Find links on this topic").click(
                        find_topic_links,
                        inputs=[file_input, topic_input],
                        outputs=doc_output
                    )
        with gr.TabItem("📅 Timetable Generator"):
            TaskRow = {
                "Task name": gr.Textbox(),
                "From (date and time input)": gr.DateTime(type="datetime", label="From (Optional)"),
                "To (date and time input)": gr.DateTime(type="datetime", label="To (Optional)"),
                "Estimated time to complete the task": gr.Textbox(placeholder="e.g., 30 minutes", label="Estimated Duration (Optional)")
            }
            task_df = gr.Dataframe(
                headers=list(TaskRow.keys()),
                datatype=["str", "str", "str", "str"],
                col_count=4,
                row_count=1, # Default single row
                label="Task List",
                interactive=True
            )
            with gr.Row():
                estimated_total_time = gr.Number(label="Estimated Total Time (e.g., 4)", value=4)
                total_time_unit = gr.Radio(["seconds", "minutes", "hours"], label="Unit", value="hours")
            timetable_output = gr.Markdown("Click 'Generate timetable' to see your schedule.", label="Generated Timetable")
            gr.Button("Generate timetable", variant="primary").click(
                generate_timetable,
                inputs=[task_df, estimated_total_time, total_time_unit],
                outputs=timetable_output
            )
if __name__ == "__main__":
    demo.launch(share=True)

%pip install google-generativeai
%pip install streamlit
%pip install PyPDF2
%pip install python-pptx
%pip install python-docx
%pip install openpyxl
%pip install pytesseract
%pip install pdf2image
!apt-get install -y tesseract-ocr poppler-utils

with open('SmartStudyAssistant.py', 'w') as f:
    f.write('''
import streamlit as SL
import google.generativeai as genai
from PIL import Image
import io
import os
from datetime import datetime
import pandas as pd
import json
from PyPDF2 import PdfReader
from pptx import Presentation
import docx
import openpyxl
import pytesseract
from pdf2image import convert_from_bytes

SL.set_page_config(
    page_title="AI_powered Smart Study Assistant",
    page_icon="🎓",
    layout="wide",
    initial_sidebar_state="expanded"
)
if 'api_keys' not in SL.session_state:
    SL.session_state.api_keys = []
if 'current_key_index' not in SL.session_state:
    SL.session_state.current_key_index = 0
if 'theme' not in SL.session_state:
    SL.session_state.theme = 'day'
if 'tasks' not in SL.session_state:
    SL.session_state.tasks = []
if 'chat_history' not in SL.session_state:
    SL.session_state.chat_history = []

def apply_theme():
    if SL.session_state.theme == 'day':
        SL.markdown("""
        <style>
        .stApp {
            background-color: #f5f5f5;
            color: #000000;
        }
        .stTextInput>div>div>input, .stTextArea>div>div>textarea {
            background-color: white;
            color: #000000;
        }
        </style>
        """, unsafe_allow_html=True)
    else:
        SL.markdown("""
        <style>
        .stApp {
            background-color: #1a1a1a;
            color: #e0e0e0;
        }
        .stTextInput>div>div>input, .stTextArea>div>div>textarea {
            background-color: #2d2d2d;
            color: #e0e0e0;
        }
        </style>
        """, unsafe_allow_html=True)
apply_theme()

class APIKeyManager:
    @staticmethod
    def get_current_key():
        if not SL.session_state.api_keys:
            return None
        return SL.session_state.api_keys[SL.session_state.current_key_index]
    @staticmethod
    def rotate_key():
        if len(SL.session_state.api_keys) > 1:
            SL.session_state.current_key_index = (SL.session_state.current_key_index + 1) % len(SL.session_state.api_keys)
            return True
        return False

def configure_gemini():
    key = APIKeyManager.get_current_key()
    if key:
        genai.configure(api_key=key)
        return genai.GenerativeModel('gemini-1.5-flash')
    return None

def extract_text_from_file(file):
    if file is None:
        return ""
    file_ext = os.path.splitext(file.name)[1].lower()
    try:
        if file_ext == '.txt':
            return file.read().decode('utf-8')
        elif file_ext == '.pdf':
            text = ""
            try:
                reader = PdfReader(io.BytesIO(file.read()))
                for page in reader.pages:
                    text += page.extract_text() + "\\n"
            except:
                pass
            if len(text.strip()) < 50:
                file.seek(0)
                images = convert_from_bytes(file.read())
                for img in images:
                    text += pytesseract.image_to_string(img) + "\\n"
            return text
        elif file_ext == '.docx' or file_ext == '.doc':
            doc = docx.Document(io.BytesIO(file.read()))
            return "\\n".join([para.text for para in doc.paragraphs])
        elif file_ext == '.pptx':
            prs = Presentation(io.BytesIO(file.read()))
            text = ""
            for slide in prs.slides:
                for shape in slide.shapes:
                    if hasattr(shape, "text"):
                        text += shape.text + "\\n"
            return text
        elif file_ext == '.xlsx':
            wb = openpyxl.load_workbook(io.BytesIO(file.read()))
            text = ""
            for sheet in wb.worksheets:
                for row in sheet.iter_rows(values_only=True):
                    text += " ".join([str(cell) for cell in row if cell]) + "\\n"
            return text
        elif file_ext in ['.jpg', '.jpeg', '.png']:
            img = Image.open(io.BytesIO(file.read()))
            return pytesseract.image_to_string(img)
    except Exception as e:
        return f"Error extracting text: {str(e)}"
    return ""

def call_gemini_with_retry(prompt, max_retries=None):
    if max_retries is None:
        max_retries = len(SL.session_state.api_keys)
    for attempt in range(max_retries):
        try:
            model = configure_gemini()
            if not model:
                return "Please configure at least one API key in the sidebar."
            response = model.generate_content(prompt)
            return response.text
        except Exception as e:
            if "quota" in str(e).lower() or "limit" in str(e).lower():
                if APIKeyManager.rotate_key():
                    continue
            if attempt == max_retries - 1:
                return f"Error: {str(e)}"
    return "All API keys exhausted. Please add more keys or try later."

with SL.sidebar:
    SL.markdown("### 🔑 API Configuration")
    api_key_inputs = []
    for i in range(10):
        key = SL.text_input(f"API Key {i+1}", type="password", key=f"api_key_{i}", placeholder="Enter Gemini API key")
        if key:
            api_key_inputs.append(key)
    if SL.button("Load API Keys"):
        SL.session_state.api_keys = [k.strip() for k in api_key_inputs if k.strip()]
        SL.session_state.current_key_index = 0
        SL.success(f"Loaded {len(SL.session_state.api_keys)} API key(s)")
    SL.markdown("---")
    SL.markdown("### 🎨 Theme")
    col1, col2 = SL.columns(2)
    with col1:
        if SL.button("☀️ Day"):
            SL.session_state.theme = 'day'
            SL.rerun()
    with col2:
        if SL.button("🌙 Night"):
            SL.session_state.theme = 'night'
            SL.rerun()

SL.title("🎓 AI Study Assistant")
tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9 = SL.tabs(["💬 Chat","📝 Summarize","📚 Explain", "🔑 Key Words","🔍 Similar Content","🌐 Web Links","❓ Q&A","🖼️ Image Search","📅 Timetable"])

with tab1:
    SL.markdown("### Chat with AI")
    for message in SL.session_state.chat_history:
        with SL.chat_message(message["role"]):
            SL.write(message["content"])
    user_input = SL.chat_input("Ask me anything...")
    if user_input:
        SL.session_state.chat_history.append({"role": "user", "content": user_input})
        with SL.chat_message("user"):
            SL.write(user_input)
        with SL.chat_message("assistant"):
            response = call_gemini_with_retry(user_input)
            SL.write(response)
            SL.session_state.chat_history.append({"role": "assistant", "content": response})

with tab2:
    SL.markdown("### Summarize Your Document")
    col1, col2 = SL.columns([2, 1])
    with col1:
        sum_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="sum_file")
    with col2:
        sum_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="sum_topic")
    if SL.button("Summarize", type="primary", key="sum_btn"):
        if sum_file:
            with SL.spinner("Generating summary..."):
                text = extract_text_from_file(sum_file)
                prompt = f"Summarize the following document"
                if sum_topic:
                    prompt += f" focusing on the topic '{sum_topic}'"
                prompt += f":\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Summary", result, height=400, key="sum_output")
        else:
            SL.warning("Please upload a document first!")

with tab3:
    SL.markdown("### Explain Document Content")
    col1, col2 = SL.columns([2, 1])
    with col1:
        exp_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="exp_file")
    with col2:
        exp_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="exp_topic")
    SL.markdown("**Select Difficulty Level:**")
    col1, col2, col3 = SL.columns(3)
    with col1:
        if SL.button("Explain for beginner level", type="primary", use_container_width=True):
            if exp_file:
                with SL.spinner("Generating explanation..."):
                    text = extract_text_from_file(exp_file)
                    prompt = f"Explain the following document at a beginner level"
                    if exp_topic:
                        prompt += f" focusing on the topic '{exp_topic}'"
                    prompt += f":\\n\\n{text}"
                    result = call_gemini_with_retry(prompt)
                    SL.text_area("Explanation", result, height=400, key="exp_beg")
            else:
                SL.warning("Please upload a document first!")
    with col2:
        if SL.button("Explain for intermediate level", type="primary", use_container_width=True):
            if exp_file:
                with SL.spinner("Generating explanation..."):
                    text = extract_text_from_file(exp_file)
                    prompt = f"Explain the following document at an intermediate level"
                    if exp_topic:
                        prompt += f" focusing on the topic '{exp_topic}'"
                    prompt += f":\\n\\n{text}"
                    result = call_gemini_with_retry(prompt)
                    SL.text_area("Explanation", result, height=400, key="exp_int")
            else:
                SL.warning("Please upload a document first!")
    with col3:
        if SL.button("Explain for advanced level", type="primary", use_container_width=True):
            if exp_file:
                with SL.spinner("Generating explanation..."):
                    text = extract_text_from_file(exp_file)
                    prompt = f"Explain the following document at an advanced level"
                    if exp_topic:
                        prompt += f" focusing on the topic '{exp_topic}'"
                    prompt += f":\\n\\n{text}"
                    result = call_gemini_with_retry(prompt)
                    SL.text_area("Explanation", result, height=400, key="exp_adv")
            else:
                SL.warning("Please upload a document first!")

with tab4:
    SL.markdown("### Find and Explain Key Words")
    col1, col2 = SL.columns([2, 1])
    with col1:
        kw_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="kw_file")
    with col2:
        kw_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="kw_topic")
    if SL.button("Find and explain key words", type="primary", key="kw_btn"):
        if kw_file:
            with SL.spinner("Finding keywords..."):
                text = extract_text_from_file(kw_file)
                prompt = f"Identify all key terms/keywords from this document"
                if kw_topic:
                    prompt += f" related to '{kw_topic}'"
                prompt += f", and explain each one with examples:\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Keywords & Explanations", result, height=400, key="kw_output")
        else:
            SL.warning("Please upload a document first!")

with tab5:
    SL.markdown("### Find Similar Documents/Images")
    col1, col2 = SL.columns([2, 1])
    with col1:
        sim_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="sim_file")
    with col2:
        sim_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="sim_topic")
    if SL.button("Find similar document/image", type="primary", key="sim_btn"):
        if sim_file:
            with SL.spinner("Searching for similar content..."):
                text = extract_text_from_file(sim_file)[:500]
                file_ext = os.path.splitext(sim_file.name)[1].lower()
                if file_ext in ['.jpg', '.jpeg', '.png']:
                    search_type = "similar image"
                else:
                    search_type = "similar document or article"
                prompt = f"Search the web for a {search_type} containing information about: {text}"
                if sim_topic:
                    prompt += f" related to '{sim_topic}'"
                prompt += "\\n\\nProvide the link and a brief description."
                result = call_gemini_with_retry(prompt)
                SL.text_area("Similar Content", result, height=400, key="sim_output")
        else:
            SL.warning("Please upload a document first!")

with tab6:
    SL.markdown("### Find Related Web Resources")
    col1, col2 = SL.columns([2, 1])
    with col1:
        link_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="link_file")
    with col2:
        link_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="link_topic")
    if SL.button("Find links", type="primary", key="link_btn"):
        if link_file:
            with SL.spinner("Searching for web links..."):
                text = extract_text_from_file(link_file)[:300]
                prompt = f"Search the web and provide at least 5 links to documentation or information sources about: {text}"
                if link_topic:
                    prompt += f" focusing on '{link_topic}'"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Web Links", result, height=400, key="link_output")
        else:
            SL.warning("Please upload a document first!")

with tab7:
    SL.markdown("### Generate Answers to Questions")
    col1, col2 = SL.columns([2, 1])
    with col1:
        qa_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="qa_file")
    with col2:
        qa_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="qa_topic")
    if SL.button("Generate answers", type="primary", key="qa_btn"):
        if qa_file:
            with SL.spinner("Generating answers..."):
                text = extract_text_from_file(qa_file)
                prompt = f"Identify all questions in this document and provide detailed answers"
                if qa_topic:
                    prompt += f" in the context of '{qa_topic}'"
                prompt += f":\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Questions & Answers", result, height=400, key="qa_output")
        else:
            SL.warning("Please upload a document first!")

with tab8:
    SL.markdown("### Find Relevant Images")
    col1, col2 = SL.columns([2, 1])
    with col1:
        img_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="img_file")
    with col2:
        img_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="img_topic")
    if SL.button("Find image", type="primary", key="img_btn"):
        if img_file:
            with SL.spinner("Searching for images..."):
                text = extract_text_from_file(img_file)[:200]
                prompt = f"Search the web for an image that matches this description: {text}"
                if img_topic:
                    prompt += f" related to '{img_topic}'"
                prompt += "\\n\\nProvide the image link and description."
                result = call_gemini_with_retry(prompt)
                SL.text_area("Image Results", result, height=400, key="img_output")
        else:
            SL.warning("Please upload a document first!")

with tab9:
    SL.markdown("### Create Your Study Timetable")
    with SL.form("task_form"):
        col1, col2, col3, col4 = SL.columns(4)
        with col1:
            task_name = SL.text_input("Task Name", placeholder="e.g., Study Math")
        with col2:
            from_datetime = SL.text_input("From (Optional)", placeholder="YYYY-MM-DD HH:MM")
        with col3:
            to_datetime = SL.text_input("To (Optional)", placeholder="YYYY-MM-DD HH:MM")
        with col4:
            est_time = SL.text_input("Estimated Time (Optional)", placeholder="e.g., 2 hours")
        add_task = SL.form_submit_button("➕ Add Task")
        if add_task and task_name:
            SL.session_state.tasks.append({
                "name": task_name,
                "from": from_datetime,
                "to": to_datetime,
                "estimated": est_time
            })
            SL.success(f"Added task: {task_name}")
    if SL.session_state.tasks:
        SL.markdown("#### Current Tasks")
        for i, task in enumerate(SL.session_state.tasks):
            SL.write(f"{i+1}. **{task['name']}** | From: {task['from']} | To: {task['to']} | Est: {task['estimated']}")
        if SL.button("Clear All Tasks"):
            SL.session_state.tasks = []
            SL.rerun()
    total_time = SL.text_input("Estimated Total Time", placeholder="e.g., 8 hours")
    if SL.button("Generate Timetable", type="primary"):
        if SL.session_state.tasks:
            with SL.spinner("Generating timetable..."):
                task_str = json.dumps(SL.session_state.tasks, indent=2)
                prompt = f"""Create a detailed timetable based on these tasks:
{task_str}
Total estimated time available: {total_time}
Generate a realistic schedule in table format with task names and time allocations."""
                result = call_gemini_with_retry(prompt)
                SL.markdown("#### Generated Timetable")
                SL.text_area("Timetable", result, height=400)
        else:
            SL.warning("Please add at least one task first!")
''')

!streamlit run SmartStudyAssistant.py & npx localtunnel --port 8501

%pip install pyngrok

from pyngrok import ngrok as ng
import subprocess
!pkill -9 streamlit

ng.set_auth_token("YOUR_AUTHTOKEN")

subprocess.Popen(["streamlit", "run", "SmartStudyAssistant.py", "--server.port", "8501", "--server.headless", "true"])
public_url = ng.connect(8501, bind_tls=True)
print(public_url)

# Step 1: Installations
!pip uninstall -y google-generativeai
!pip install -q google-genai streamlit PyPDF2 python-pptx python-docx openpyxl Pillow pytesseract pdf2image pyngrok
!apt-get install -y tesseract-ocr poppler-utils > /dev/null 2>&1

# Step 2: Run the code above to create SmartStudyAssistant.py

# Step 3: Start the app
!streamlit run SmartStudyAssistant.py & npx localtunnel --port 8501

%pip install google-genai
%pip install streamlit
%pip install PyPDF2
%pip install python-pptx
%pip install python-docx
%pip install openpyxl
%pip install Pillow
%pip install pytesseract
%pip install pdf2image
!apt-get install -y tesseract-ocr poppler-utils > /dev/null 2>&1

with open('SmartStudyAssistant.py', 'w') as f:
    f.write('''
import streamlit as SL
from google import genai
from google.genai import types
from PIL import Image
import io
import os
from datetime import datetime
import pandas as pd
import json
from PyPDF2 import PdfReader
from pptx import Presentation
import docx
import openpyxl
import pytesseract
from pdf2image import convert_from_bytes

SL.set_page_config(
    page_title="AI_powered Smart Study Assistant",
    page_icon="🎓",
    layout="wide",
    initial_sidebar_state="expanded"
)
if 'api_keys' not in SL.session_state:
    SL.session_state.api_keys = []
if 'current_key_index' not in SL.session_state:
    SL.session_state.current_key_index = 0
if 'theme' not in SL.session_state:
    SL.session_state.theme = 'day'
if 'tasks' not in SL.session_state:
    SL.session_state.tasks = []
if 'chat_history' not in SL.session_state:
    SL.session_state.chat_history = []
if 'client' not in SL.session_state:
    SL.session_state.client = None

def apply_theme():
    if SL.session_state.theme == 'day':
        SL.markdown("""
        <style>
        .stApp {
            background-color: #f5f5f5;
            color: #000000;
        }
        .stTextInput>div>div>input, .stTextArea>div>div>textarea {
            background-color: white;
            color: #000000;
        }
        </style>
        """, unsafe_allow_html=True)
    else:
        SL.markdown("""
        <style>
        .stApp {
            background-color: #1a1a1a;
            color: #e0e0e0;
        }
        .stTextInput>div>div>input, .stTextArea>div>div>textarea {
            background-color: #2d2d2d;
            color: #e0e0e0;
        }
        </style>
        """, unsafe_allow_html=True)
apply_theme()

class APIKeyManager:
    @staticmethod
    def get_current_key():
        if not SL.session_state.api_keys:
            return None
        return SL.session_state.api_keys[SL.session_state.current_key_index]
    @staticmethod
    def rotate_key():
        if len(SL.session_state.api_keys) > 1:
            SL.session_state.current_key_index = (SL.session_state.current_key_index + 1) % len(SL.session_state.api_keys)
            return True
        return False

def configure_gemini():
    key = APIKeyManager.get_current_key()
    if key:
        try:
            client = genai.Client(api_key=key)
            SL.session_state.client = client
            return client
        except Exception as e:
            return None
    return None

def extract_text_from_file(file):
    if file is None:
        return ""
    file_ext = os.path.splitext(file.name)[1].lower()
    try:
        if file_ext == '.txt':
            return file.read().decode('utf-8')
        elif file_ext == '.pdf':
            text = ""
            try:
                reader = PdfReader(io.BytesIO(file.read()))
                for page in reader.pages:
                    text += page.extract_text() + "\\n"
            except:
                pass
            if len(text.strip()) < 50:
                file.seek(0)
                images = convert_from_bytes(file.read())
                for img in images:
                    text += pytesseract.image_to_string(img) + "\\n"
            return text
        elif file_ext == '.docx' or file_ext == '.doc':
            doc = docx.Document(io.BytesIO(file.read()))
            return "\\n".join([para.text for para in doc.paragraphs])
        elif file_ext == '.pptx':
            prs = Presentation(io.BytesIO(file.read()))
            text = ""
            for slide in prs.slides:
                for shape in slide.shapes:
                    if hasattr(shape, "text"):
                        text += shape.text + "\\n"
            return text
        elif file_ext == '.xlsx':
            wb = openpyxl.load_workbook(io.BytesIO(file.read()))
            text = ""
            for sheet in wb.worksheets:
                for row in sheet.iter_rows(values_only=True):
                    text += " ".join([str(cell) for cell in row if cell]) + "\\n"
            return text
        elif file_ext in ['.jpg', '.jpeg', '.png']:
            img = Image.open(io.BytesIO(file.read()))
            return pytesseract.image_to_string(img)
    except Exception as e:
        return f"Error extracting text: {str(e)}"
    return ""

def call_gemini_with_retry(prompt, max_retries=None):
    if max_retries is None:
        max_retries = max(len(SL.session_state.api_keys), 1)
    for attempt in range(max_retries):
        try:
            client = configure_gemini()
            if not client:
                return "Please configure at least one API key in the sidebar."
            response = client.models.generate_content(
                model='gemini-2.0-flash-exp',
                contents=prompt
            )
            return response.text
        except Exception as e:
            error_str = str(e).lower()
            if "quota" in error_str or "limit" in error_str or "resource_exhausted" in error_str:
                if APIKeyManager.rotate_key():
                    continue
            if attempt == max_retries - 1:
                return f"Error: {str(e)}"
    return "All API keys exhausted. Please add more keys or try later."

with SL.sidebar:
    SL.markdown("### 🔑 API Configuration")
    api_key_inputs = []
    for i in range(10):
        key = SL.text_input(f"API Key {i+1}", type="password", key=f"api_key_{i}", placeholder="Enter Gemini API key")
        if key:
            api_key_inputs.append(key)
    if SL.button("Load API Keys"):
        SL.session_state.api_keys = [k.strip() for k in api_key_inputs if k.strip()]
        SL.session_state.current_key_index = 0
        SL.success(f"Loaded {len(SL.session_state.api_keys)} API key(s)")
    SL.markdown("---")
    SL.markdown("### 🎨 Theme")
    col1, col2 = SL.columns(2)
    with col1:
        if SL.button("☀️ Day"):
            SL.session_state.theme = 'day'
            SL.rerun()
    with col2:
        if SL.button("🌙 Night"):
            SL.session_state.theme = 'night'
            SL.rerun()

SL.title("🎓 AI Study Assistant")
tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9 = SL.tabs(["💬 Chat","📝 Summarize","📚 Explain", "🔑 Key Words","🔍 Similar Content","🌐 Web Links","❓ Q&A","🖼️ Image Search","📅 Timetable"])

with tab1:
    SL.markdown("### Chat with AI")
    for message in SL.session_state.chat_history:
        with SL.chat_message(message["role"]):
            SL.write(message["content"])
    user_input = SL.chat_input("Ask me anything...")
    if user_input:
        SL.session_state.chat_history.append({"role": "user", "content": user_input})
        with SL.chat_message("user"):
            SL.write(user_input)
        with SL.chat_message("assistant"):
            response = call_gemini_with_retry(user_input)
            SL.write(response)
            SL.session_state.chat_history.append({"role": "assistant", "content": response})

with tab2:
    SL.markdown("### Summarize Your Document")
    col1, col2 = SL.columns([2, 1])
    with col1:
        sum_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="sum_file")
    with col2:
        sum_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="sum_topic")
    if SL.button("Summarize", type="primary", key="sum_btn"):
        if sum_file:
            with SL.spinner("Generating summary..."):
                text = extract_text_from_file(sum_file)
                prompt = f"Summarize the following document"
                if sum_topic:
                    prompt += f" focusing on the topic '{sum_topic}'"
                prompt += f":\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Summary", result, height=400, key="sum_output")
        else:
            SL.warning("Please upload a document first!")

with tab3:
    SL.markdown("### Explain Document Content")
    col1, col2 = SL.columns([2, 1])
    with col1:
        exp_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="exp_file")
    with col2:
        exp_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="exp_topic")
    SL.markdown("**Select Difficulty Level:**")
    col1, col2, col3 = SL.columns(3)
    with col1:
        if SL.button("Explain for beginner level", type="primary", use_container_width=True):
            if exp_file:
                with SL.spinner("Generating explanation..."):
                    text = extract_text_from_file(exp_file)
                    prompt = f"Explain the following document at a beginner level"
                    if exp_topic:
                        prompt += f" focusing on the topic '{exp_topic}'"
                    prompt += f":\\n\\n{text}"
                    result = call_gemini_with_retry(prompt)
                    SL.text_area("Explanation", result, height=400, key="exp_beg")
            else:
                SL.warning("Please upload a document first!")
    with col2:
        if SL.button("Explain for intermediate level", type="primary", use_container_width=True):
            if exp_file:
                with SL.spinner("Generating explanation..."):
                    text = extract_text_from_file(exp_file)
                    prompt = f"Explain the following document at an intermediate level"
                    if exp_topic:
                        prompt += f" focusing on the topic '{exp_topic}'"
                    prompt += f":\\n\\n{text}"
                    result = call_gemini_with_retry(prompt)
                    SL.text_area("Explanation", result, height=400, key="exp_int")
            else:
                SL.warning("Please upload a document first!")
    with col3:
        if SL.button("Explain for advanced level", type="primary", use_container_width=True):
            if exp_file:
                with SL.spinner("Generating explanation..."):
                    text = extract_text_from_file(exp_file)
                    prompt = f"Explain the following document at an advanced level"
                    if exp_topic:
                        prompt += f" focusing on the topic '{exp_topic}'"
                    prompt += f":\\n\\n{text}"
                    result = call_gemini_with_retry(prompt)
                    SL.text_area("Explanation", result, height=400, key="exp_adv")
            else:
                SL.warning("Please upload a document first!")

with tab4:
    SL.markdown("### Find and Explain Key Words")
    col1, col2 = SL.columns([2, 1])
    with col1:
        kw_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="kw_file")
    with col2:
        kw_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="kw_topic")
    if SL.button("Find and explain key words", type="primary", key="kw_btn"):
        if kw_file:
            with SL.spinner("Finding keywords..."):
                text = extract_text_from_file(kw_file)
                prompt = f"Identify all key terms/keywords from this document"
                if kw_topic:
                    prompt += f" related to '{kw_topic}'"
                prompt += f", and explain each one with examples:\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Keywords & Explanations", result, height=400, key="kw_output")
        else:
            SL.warning("Please upload a document first!")

with tab5:
    SL.markdown("### Find Similar Documents/Images")
    col1, col2 = SL.columns([2, 1])
    with col1:
        sim_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="sim_file")
    with col2:
        sim_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="sim_topic")
    if SL.button("Find similar document/image", type="primary", key="sim_btn"):
        if sim_file:
            with SL.spinner("Searching for similar content..."):
                text = extract_text_from_file(sim_file)[:500]
                file_ext = os.path.splitext(sim_file.name)[1].lower()
                if file_ext in ['.jpg', '.jpeg', '.png']:
                    search_type = "similar image"
                else:
                    search_type = "similar document or article"
                prompt = f"Search the web for a {search_type} containing information about: {text}"
                if sim_topic:
                    prompt += f" related to '{sim_topic}'"
                prompt += "\\n\\nProvide the link and a brief description."
                result = call_gemini_with_retry(prompt)
                SL.text_area("Similar Content", result, height=400, key="sim_output")
        else:
            SL.warning("Please upload a document first!")

with tab6:
    SL.markdown("### Find Related Web Resources")
    col1, col2 = SL.columns([2, 1])
    with col1:
        link_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="link_file")
    with col2:
        link_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="link_topic")
    if SL.button("Find links", type="primary", key="link_btn"):
        if link_file:
            with SL.spinner("Searching for web links..."):
                text = extract_text_from_file(link_file)[:300]
                prompt = f"Search the web and provide at least 5 links to documentation or information sources about: {text}"
                if link_topic:
                    prompt += f" focusing on '{link_topic}'"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Web Links", result, height=400, key="link_output")
        else:
            SL.warning("Please upload a document first!")

with tab7:
    SL.markdown("### Generate Answers to Questions")
    col1, col2 = SL.columns([2, 1])
    with col1:
        qa_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="qa_file")
    with col2:
        qa_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="qa_topic")
    if SL.button("Generate answers", type="primary", key="qa_btn"):
        if qa_file:
            with SL.spinner("Generating answers..."):
                text = extract_text_from_file(qa_file)
                prompt = f"Identify all questions in this document and provide detailed answers"
                if qa_topic:
                    prompt += f" in the context of '{qa_topic}'"
                prompt += f":\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Questions & Answers", result, height=400, key="qa_output")
        else:
            SL.warning("Please upload a document first!")

with tab8:
    SL.markdown("### Find Relevant Images")
    col1, col2 = SL.columns([2, 1])
    with col1:
        img_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="img_file")
    with col2:
        img_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="img_topic")
    if SL.button("Find image", type="primary", key="img_btn"):
        if img_file:
            with SL.spinner("Searching for images..."):
                text = extract_text_from_file(img_file)[:200]
                prompt = f"Search the web for an image that matches this description: {text}"
                if img_topic:
                    prompt += f" related to '{img_topic}'"
                prompt += "\\n\\nProvide the image link and description."
                result = call_gemini_with_retry(prompt)
                SL.text_area("Image Results", result, height=400, key="img_output")
        else:
            SL.warning("Please upload a document first!")

with tab9:
    SL.markdown("### Create Your Study Timetable")
    with SL.form("task_form"):
        col1, col2, col3, col4 = SL.columns(4)
        with col1:
            task_name = SL.text_input("Task Name", placeholder="e.g., Study Math")
        with col2:
            from_datetime = SL.text_input("From (Optional)", placeholder="YYYY-MM-DD HH:MM")
        with col3:
            to_datetime = SL.text_input("To (Optional)", placeholder="YYYY-MM-DD HH:MM")
        with col4:
            est_time = SL.text_input("Estimated Time (Optional)", placeholder="e.g., 2 hours")
        add_task = SL.form_submit_button("➕ Add Task")
        if add_task and task_name:
            SL.session_state.tasks.append({
                "name": task_name,
                "from": from_datetime,
                "to": to_datetime,
                "estimated": est_time
            })
            SL.success(f"Added task: {task_name}")
    if SL.session_state.tasks:
        SL.markdown("#### Current Tasks")
        for i, task in enumerate(SL.session_state.tasks):
            SL.write(f"{i+1}. **{task['name']}** | From: {task['from']} | To: {task['to']} | Est: {task['estimated']}")
        if SL.button("Clear All Tasks"):
            SL.session_state.tasks = []
            SL.rerun()
    total_time = SL.text_input("Estimated Total Time", placeholder="e.g., 8 hours")
    if SL.button("Generate Timetable", type="primary"):
        if SL.session_state.tasks:
            with SL.spinner("Generating timetable..."):
                task_str = json.dumps(SL.session_state.tasks, indent=2)
                prompt = f"""Create a detailed timetable based on these tasks:
{task_str}
Total estimated time available: {total_time}
Generate a realistic schedule in table format with task names and time allocations."""
                result = call_gemini_with_retry(prompt)
                SL.markdown("#### Generated Timetable")
                SL.text_area("Timetable", result, height=400)
        else:
            SL.warning("Please add at least one task first!")
''')

!streamlit run SmartStudyAssistant.py & npx localtunnel --port 8501

In [ ]:
%pip install google-genai
%pip install streamlit
%pip install PyPDF2
%pip install python-pptx
%pip install python-docx
%pip install openpyxl
%pip install Pillow
%pip install pytesseract
%pip install pdf2image
!apt-get install -y tesseract-ocr poppler-utils > /dev/null 2>&1

In [ ]:
!mkdir -p stdast
!mkdir -p stdast/utils
!mkdir -p stdast/tabs

In [ ]:
with open('stdast/utils/APIs.py', 'w') as f1:
    f1.write('''
from google import genai
import streamlit as SL
class APIKeyManager:
    @staticmethod
    def get_current_key():
        if not SL.session_state.api_keys:
            return None
        return SL.session_state.api_keys[SL.session_state.current_key_index]
    @staticmethod
    def rotate_key():
        if len(SL.session_state.api_keys) > 1:
            SL.session_state.current_key_index = (SL.session_state.current_key_index + 1) % len(SL.session_state.api_keys)
            return True
        return False
def configure_gemini():
    key = APIKeyManager.get_current_key()
    if key:
        try:
            client = genai.Client(api_key=key)
            SL.session_state.client = client
            return client
        except Exception as e:
            return None
    return None
def call_gemini_with_retry(prompt, max_retries=None):
    if max_retries is None:
        max_retries = max(len(SL.session_state.api_keys), 1)
    for attempt in range(max_retries):
        try:
            client = configure_gemini()
            if not client:
                return "Please configure at least one API key in the sidebar."
            response = client.models.generate_content(
                model='gemini-2.0-flash-exp',
                contents=prompt
            )
            return response.text
        except Exception as e:
            error_str = str(e).lower()
            if "quota" in error_str or "limit" in error_str or "resource_exhausted" in error_str:
                if APIKeyManager.rotate_key():
                    continue
            if attempt == max_retries - 1:
                return f"Error: {str(e)}"
    return "All API keys exhausted. Please add more keys or try later."
''')

In [ ]:
with open('stdast/utils/DOCs.py', 'w') as f2:
    f2.write('''
import io
import os
from PIL import Image
from PyPDF2 import PdfReader
from pptx import Presentation
import docx
import openpyxl
import pytesseract
from pdf2image import convert_from_bytes
def extract_text_from_file(file):
    if file is None:
        return ""
    file_ext = os.path.splitext(file.name)[1].lower()
    try:
        if file_ext == '.txt':
            return file.read().decode('utf-8')
        elif file_ext == '.pdf':
            text = ""
            try:
                reader = PdfReader(io.BytesIO(file.read()))
                for page in reader.pages:
                    text += page.extract_text() + "\\n"
            except:
                pass
            if len(text.strip()) < 50:
                file.seek(0)
                images = convert_from_bytes(file.read())
                for img in images:
                    text += pytesseract.image_to_string(img) + "\\n"
            return text
        elif file_ext == '.docx' or file_ext == '.doc':
            doc = docx.Document(io.BytesIO(file.read()))
            return "\\n".join([para.text for para in doc.paragraphs])
        elif file_ext == '.pptx':
            prs = Presentation(io.BytesIO(file.read()))
            text = ""
            for slide in prs.slides:
                for shape in slide.shapes:
                    if hasattr(shape, "text"):
                        text += shape.text + "\\n"
            return text
        elif file_ext == '.xlsx':
            wb = openpyxl.load_workbook(io.BytesIO(file.read()))
            text = ""
            for sheet in wb.worksheets:
                for row in sheet.iter_rows(values_only=True):
                    text += " ".join([str(cell) for cell in row if cell]) + "\\n"
            return text
        elif file_ext in ['.jpg', '.jpeg', '.png']:
            img = Image.open(io.BytesIO(file.read()))
            return pytesseract.image_to_string(img)
    except Exception as e:
        return f"Error extracting text: {str(e)}"
    return ""
''')

In [ ]:
with open('stdast/utils/Theme.py', 'w') as f3:
    f3.write('''
import streamlit as SL
def apply_theme():
    if SL.session_state.theme == 'day':
        SL.markdown("""
        <style>
        .stApp {
            background-color: #f5f5f5;
            color: #000000;
        }
        .stTextInput>div>div>input, .stTextArea>div>div>textarea {
            background-color: white;
            color: #000000;
        }
        </style>
        """, unsafe_allow_html=True)
    else:
        SL.markdown("""
        <style>
        .stApp {
            background-color: #1a1a1a;
            color: #e0e0e0;
        }
        .stTextInput>div>div>input, .stTextArea>div>div>textarea {
            background-color: #2d2d2d;
            color: #e0e0e0;
        }
        </style>
        """, unsafe_allow_html=True)
''')

In [ ]:
with open('stdast/tabs/Chat.py', 'w') as f4:
    f4.write('''
import streamlit as SL
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
def render():
    SL.markdown("### Chat with AI")
    for message in SL.session_state.chat_history:
        with SL.chat_message(message["role"]):
            SL.write(message["content"])
    user_input = SL.chat_input("Ask me anything...")
    if user_input:
        SL.session_state.chat_history.append({"role": "user", "content": user_input})
        with SL.chat_message("user"):
            SL.write(user_input)
        with SL.chat_message("assistant"):
            response = call_gemini_with_retry(user_input)
            SL.write(response)
            SL.session_state.chat_history.append({"role": "assistant", "content": response})
''')

In [ ]:
with open('stdast/tabs/Summ.py', 'w') as f5:
    f5.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_summarize():
    SL.markdown("### Summarize Your Document")
    col1, col2 = SL.columns([2, 1])
    with col1:
        sum_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="sum_file")
    with col2:
        sum_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="sum_topic")
    if SL.button("Summarize", type="primary", key="sum_btn"):
        if sum_file:
            with SL.spinner("Generating summary..."):
                text = extract_text_from_file(sum_file)
                prompt = f"Summarize the following document"
                if sum_topic:
                    prompt += f" focusing on the topic '{sum_topic}'"
                prompt += f":\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Summary", result, height=400, key="sum_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/Exp.py', 'w') as f6:
    f6.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_explain():
    SL.markdown("### Explain Document Content")
    col1, col2 = SL.columns([2, 1])
    with col1:
        exp_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="exp_file")
    with col2:
        exp_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="exp_topic")
    SL.markdown("**Select Difficulty Level:**")
    col1, col2, col3 = SL.columns(3)
    levels = [("beginner", col1, "exp_beg"), ("intermediate", col2, "exp_int"), ("advanced", col3, "exp_adv")]
    for level, col, key in levels:
        with col:
            if SL.button(f"Explain for {level} level", type="primary", use_container_width=True, key=f"btn_{key}"):
                if exp_file:
                    with SL.spinner("Generating explanation..."):
                        text = extract_text_from_file(exp_file)
                        prompt = f"Explain the following document at a {level} level"
                        if exp_topic:
                            prompt += f" focusing on the topic '{exp_topic}'"
                        prompt += f":\\n\\n{text}"
                        result = call_gemini_with_retry(prompt)
                        SL.text_area("Explanation", result, height=400, key=key)
                else:
                    SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/KeyW.py', 'w') as f7:
    f7.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_keywords():
    SL.markdown("### Find and Explain Key Words")
    col1, col2 = SL.columns([2, 1])
    with col1:
        kw_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="kw_file")
    with col2:
        kw_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="kw_topic")
    if SL.button("Find and explain key words", type="primary", key="kw_btn"):
        if kw_file:
            with SL.spinner("Finding keywords..."):
                text = extract_text_from_file(kw_file)
                prompt = f"Identify all key terms/keywords from this document"
                if kw_topic:
                    prompt += f" related to '{kw_topic}'"
                prompt += f", and explain each one with examples:\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Keywords & Explanations", result, height=400, key="kw_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/Sim.py', 'w') as f8:
    f8.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_similar():
    SL.markdown("### Find Similar Documents/Images")
    col1, col2 = SL.columns([2, 1])
    with col1:
        sim_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="sim_file")
    with col2:
        sim_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="sim_topic")
    if SL.button("Find similar document/image", type="primary", key="sim_btn"):
        if sim_file:
            with SL.spinner("Searching for similar content..."):
                text = extract_text_from_file(sim_file)[:500]
                file_ext = os.path.splitext(sim_file.name)[1].lower()
                search_type = "similar image" if file_ext in ['.jpg', '.jpeg', '.png'] else "similar document or article"
                prompt = f"Search the web for a {search_type} containing information about: {text}"
                if sim_topic:
                    prompt += f" related to '{sim_topic}'"
                prompt += "\\n\\nProvide the link and a brief description."
                result = call_gemini_with_retry(prompt)
                SL.text_area("Similar Content", result, height=400, key="sim_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/Links.py', 'w') as f9:
    f9.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_links():
    SL.markdown("### Find Related Web Resources")
    col1, col2 = SL.columns([2, 1])
    with col1:
        link_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="link_file")
    with col2:
        link_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="link_topic")
    if SL.button("Find links", type="primary", key="link_btn"):
        if link_file:
            with SL.spinner("Searching for web links..."):
                text = extract_text_from_file(link_file)[:300]
                prompt = f"Search the web and provide at least 5 links to documentation or information sources about: {text}"
                if link_topic:
                    prompt += f" focusing on '{link_topic}'"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Web Links", result, height=400, key="link_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/QA.py', 'w') as f10:
    f10.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_qa():
    SL.markdown("### Generate Answers to Questions")
    col1, col2 = SL.columns([2, 1])
    with col1:
        qa_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="qa_file")
    with col2:
        qa_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="qa_topic")
    if SL.button("Generate answers", type="primary", key="qa_btn"):
        if qa_file:
            with SL.spinner("Generating answers..."):
                text = extract_text_from_file(qa_file)
                prompt = f"Identify all questions in this document and provide detailed answers"
                if qa_topic:
                    prompt += f" in the context of '{qa_topic}'"
                prompt += f":\\n\\n{text}"
                result = call_gemini_with_retry(prompt)
                SL.text_area("Questions & Answers", result, height=400, key="qa_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/Img.py', 'w') as f11:
    f11.write('''
import streamlit as SL
import os
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
from utils.DOCs import extract_text_from_file
def render_image_search():
    SL.markdown("### Find Relevant Images")
    col1, col2 = SL.columns([2, 1])
    with col1:
        img_file = SL.file_uploader("Upload Document", type=['txt', 'pdf', 'docx', 'doc', 'pptx', 'jpg', 'jpeg', 'png', 'xlsx'], key="img_file")
    with col2:
        img_topic = SL.text_input("Topic Name (Optional)", placeholder="Enter the topic name", key="img_topic")
    if SL.button("Find image", type="primary", key="img_btn"):
        if img_file:
            with SL.spinner("Searching for images..."):
                text = extract_text_from_file(img_file)[:200]
                prompt = f"Search the web for an image that matches this description: {text}"
                if img_topic:
                    prompt += f" related to '{img_topic}'"
                prompt += "\\n\\nProvide the image link and description."
                result = call_gemini_with_retry(prompt)
                SL.text_area("Image Results", result, height=400, key="img_output")
        else:
            SL.warning("Please upload a document first!")
''')

In [ ]:
with open('stdast/tabs/TimTab.py', 'w') as f12:
    f12.write('''
import streamlit as SL
import json
import sys
sys.path.append('/content/stdast')
from utils.APIs import call_gemini_with_retry
def render():
    SL.markdown("### Create Your Study Timetable")
    with SL.form("task_form"):
        col1, col2, col3, col4 = SL.columns(4)
        with col1:
            task_name = SL.text_input("Task Name", placeholder="e.g., Study Math")
        with col2:
            from_datetime = SL.text_input("From (Optional)", placeholder="YYYY-MM-DD HH:MM")
        with col3:
            to_datetime = SL.text_input("To (Optional)", placeholder="YYYY-MM-DD HH:MM")
        with col4:
            est_time = SL.text_input("Estimated Time (Optional)", placeholder="e.g., 2 hours")
        add_task = SL.form_submit_button("➕ Add Task")
        if add_task and task_name:
            SL.session_state.tasks.append({
                "name": task_name,
                "from": from_datetime,
                "to": to_datetime,
                "estimated": est_time
            })
            SL.success(f"Added task: {task_name}")
    if SL.session_state.tasks:
        SL.markdown("#### Current Tasks")
        for i, task in enumerate(SL.session_state.tasks):
            SL.write(f"{i+1}. **{task['name']}** | From: {task['from']} | To: {task['to']} | Est: {task['estimated']}")
        if SL.button("Clear All Tasks"):
            SL.session_state.tasks = []
            SL.rerun()
    total_time = SL.text_input("Estimated Total Time", placeholder="e.g., 8 hours")
    if SL.button("Generate Timetable", type="primary"):
        if SL.session_state.tasks:
            with SL.spinner("Generating timetable..."):
                task_str = json.dumps(SL.session_state.tasks, indent=2)
                prompt = f"""Create a detailed timetable based on these tasks:
{task_str}
Total estimated time available: {total_time}
Generate a realistic schedule in table format with task names and time allocations."""
                result = call_gemini_with_retry(prompt)
                SL.markdown("#### Generated Timetable")
                SL.text_area("Timetable", result, height=400)
        else:
            SL.warning("Please add at least one task first!")
''')

In [ ]:
with open('stdast/SmartStudyAssistant.py', 'w') as f:
    f.write('''
import streamlit as SL
import sys
sys.path.append('/content/stdast')
from utils.Theme import apply_theme
from tabs import Chat, Summ, Exp, KeyW, Sim, Links, QA, Img, TimTab
SL.set_page_config(
    page_title="AI Study Assistant",
    page_icon="🎓",
    layout="wide",
    initial_sidebar_state="expanded"
)
if 'api_keys' not in SL.session_state:
    SL.session_state.api_keys = []
if 'current_key_index' not in SL.session_state:
    SL.session_state.current_key_index = 0
if 'theme' not in SL.session_state:
    SL.session_state.theme = 'day'
if 'tasks' not in SL.session_state:
    SL.session_state.tasks = []
if 'chat_history' not in SL.session_state:
    SL.session_state.chat_history = []
if 'client' not in SL.session_state:
    SL.session_state.client = None
apply_theme()
with SL.sidebar:
    SL.markdown("### 🔑 API Configuration")
    api_key_inputs = []
    for i in range(10):
        key = SL.text_input(f"API Key {i+1}", type="password", key=f"api_key_{i}", placeholder="Enter Gemini API key")
        if key:
            api_key_inputs.append(key)
    if SL.button("Load API Keys"):
        SL.session_state.api_keys = [k.strip() for k in api_key_inputs if k.strip()]
        SL.session_state.current_key_index = 0
        SL.success(f"Loaded {len(SL.session_state.api_keys)} API key(s)")
    SL.markdown("---")
    SL.markdown("### 🎨 Theme")
    col1, col2 = SL.columns(2)
    with col1:
        if SL.button("☀️ Day"):
            SL.session_state.theme = 'day'
            SL.rerun()
    with col2:
        if SL.button("🌙 Night"):
            SL.session_state.theme = 'night'
            SL.rerun()
SL.title("🎓 AI-powered Smart Study Assistant")
tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9 = SL.tabs(["💬 Chat","📝 Summarize","📚 Explain","🔑 Key Words","🔍 Similar Content","🌐 Web Links","❓ Q&A","🖼️ Image Search","📅 Timetable"])
with tab1:
    Chat.render()
with tab2:
    Summ.render_summarize()
with tab3:
    Exp.render_explain()
with tab4:
    KeyW.render_keywords()
with tab5:
    Sim.render_similar()
with tab6:
    Links.render_links()
with tab7:
    QA.render_qa()
with tab8:
    Img.render_image_search()
with tab9:
    TimTab.render()
''')

In [ ]:
!touch stdast/__init__.py
!touch stdast/utils/__init__.py
!touch stdast/tabs/__init__.py

In [ ]:
!streamlit run stdast/SmartStudyAssistant.py & npx localtunnel --port 8501